In [1]:
import requests
from bs4 import BeautifulSoup
import tqdm
import scrapy
import click

In [2]:
import time
import random
import jsonlines
import json

In [3]:
url = "https://www.ptt.cc/bbs/Beauty/index.html"
reg = requests.Session()
payload = {
'yes': 'yes'
}
reg.post('https://www.ptt.cc/ask/over18', data=payload)
prefix = "https://www.ptt.cc"

# 1.Crawl

In [4]:
flag = 0

url = "https://www.ptt.cc/bbs/Beauty/index.html"
articles_titles = []
articles_url = []
articles_hls = []
articles_dates = []

while flag == 0 or flag == 1:
    res = reg.get(url)
    content = res.text
    soup = BeautifulSoup(content, "html.parser")
    articles_title = soup.find_all(class_ = "title")
    next_page = soup.find_all(class_ = "btn wide")[1].get('href')
    url = prefix+next_page
    articles = soup.find_all(class_ = "r-ent")
    #print(articles)
    for article in articles:
        if len(article.find_all(class_ = "title")[0].find_all('a'))!= 0:
            title = article.find_all(class_ = "title")[0].find_all('a')[0].string
            if(title.find('公告')== -1):
                art_url = article.find_all(class_ = "title")[0].find_all('a')[0].get('href')
                date = article.find_all(class_ = "date")[0].text
                nrec = article.find_all(class_ = "nrec")
                pop = nrec[0].text if nrec[0].text != '' else '0'
                print(f'{title} {date} {pop}')
                articles_titles.append(title)
                articles_dates.append(date)
                articles_url.append(art_url)
                articles_hls.append(pop)
                
                if date ==  '12/31':
                    flag = 1 if len(articles_titles)< 3000 else -1
    print(next_page)
    d =random.uniform(0.5, 0.9)
    time.sleep(d)

[神人] ig髮型設計廣告  3/06 0
/bbs/Beauty/index4001.html
[正妹] aespa Karina 劉知珉  3/05 34
[正妹] SISTAR 尹寶拉  3/05 12
[正妹] Lorraine Michaels 美國模特兒  3/05 0
[正妹] Sapir Elgrabli 以色列女孩  3/05 13
[神人] 好想放假去海灘啊啊啊  3/05 8
[正妹] Cosplay 158 韓國 迪士尼  3/05 2
[正妹] 六年級  3/05 54
[正妹] 拉筋  3/05 9
[正妹] Oh My Girl YooA  3/05 5
[正妹] 斉藤里奈  3/06 4
[正妹] Cosplay 159 日本 雛田 四葉  3/06 0
[正妹] 辛樹芬  3/06 14
[正妹] 可愛  3/06 13
[正妹] 臺灣 hsin yu liao  3/06 1
[正妹] Cosplay 160 日本 應該撐不了幾分鐘  3/06 2
[正妹] 日向坂46 小西 夏菜実  3/06 0
/bbs/Beauty/index4000.html
[正妹] Ella Freya 惡靈古堡4臉模  3/04 27
[正妹] 給我看看  3/05 9
[正妹] 一日拉麵女老闆Cosplay  3/05 33
[正妹] Cosplay 156 日本 盔甲  3/05 2
[正妹] 方季惟  3/05 23
[神人] FB上很洗腦的廣告女主  3/05 3
[正妹] 再來一次 老了但還是很讚吧  3/05 81
[正妹] 久保史緒里 開球  3/05 4
[正妹] Cosplay 157 日本 齁  3/05 0
[正妹] Elizaveta Lisova 俄羅斯模特兒  3/05 2
[正妹] 騎乗位の天才 Miru  3/05 28
[正妹] 腋下  3/05 8
[神人] 富邦熱身賽看台球迷  3/05 10
[神人] 醫生朋友  3/05 11
[正妹] 馬來西亞 牙醫 溫美質  3/05 2
[正妹] 高允貞  3/05 8
[正妹] 崔有真  3/05 3
[正妹] Dreamcatcher JiU 祉攸  3/05 9
/bbs/Beauty/index3999.html
[正妹] 日本熟女創作者  3/03 4
[正

In [5]:
articles_year = []

for i,d in enumerate(articles_dates):
    month, day = d.split('/')
    if i < 3000:
        if  int(month) < 6:
            articles_year.append(2023)
        else:
            articles_year.append(2022)
    else:
        if int(month) < 12:
            articles_year.append(2022)
        else:
            articles_year.append(2021)

In [6]:
titles_2022 = []
dates_2022 = []
url_2022 = []
hl_2022 = []
for i in range(len(articles_url)):
    if articles_year[i] == 2022:
        titles_2022.append(articles_titles[i])
        hl_2022.append(articles_hls[i])
        url_2022.append(prefix+articles_url[i])
        month, day = articles_dates[i].split('/')
        tmp = ''
        if month[0] == ' ':
            tmp = '0'+month[1]+ day
        else:
            tmp = month+ day
        dates_2022.append(tmp)

In [9]:
with jsonlines.open("all_non_popular.jsonl", mode='w') as w:
    for i in range(len(url_2022)):
        if hl_2022[i] != '爆':
            line = {
                "date": dates_2022[i],
                "title": titles_2022[i],
                "url": url_2022[i],
            }

            w.write(line)

In [10]:
with jsonlines.open("all_article.jsonl", mode='w') as w:
    for i in range(len(url_2022)):
            line = {
                "date": dates_2022[i],
                "title": titles_2022[i],
                "url": url_2022[i],
            }

            w.write(line)

with jsonlines.open("all_popular.jsonl", mode='w') as w:
    for i in range(len(url_2022)):
        if hl_2022[i] == '爆':
            line = {
                "date": dates_2022[i],
                "title": titles_2022[i],
                "url": url_2022[i],
            }

            w.write(line)

In [10]:
with jsonlines.open("all_pop_2.jsonl", mode='w') as w:
    for i in range(len(url_2022)):
        if hl_2022[i][0]!='X':
            if hl_2022[i] == '爆' or int(hl_2022[i])>35:
                line = {
                    "date": dates_2022[i],
                    "title": titles_2022[i],
                    "url": url_2022[i],
                }
                w.write(line)

In [18]:
with jsonlines.open("all_non_pop_2.jsonl", mode='w') as w:
    for i in range(len(url_2022)):
        if hl_2022[i][0]!='X':
            if hl_2022[i] != '爆' and int(hl_2022[i])<35:
                line = {
                    "date": dates_2022[i],
                    "title": titles_2022[i],
                    "url": url_2022[i],
                }
                w.write(line)
                w.write('\n')

In [48]:
for i in range(len(url_2022)):
    if hl_2022[i][0]!='X':
        if hl_2022[i] != '爆' and int(hl_2022[i])<35:
            line = {
                "date": dates_2022[i],"title": titles_2022[i],"url": url_2022[i],
            }
            with open("test.jsonl", 'a') as f:
                f.write(json.dumps(line, ensure_ascii=False)+ '\n')

In [51]:
with open('all_article.jsonl','r') as f:
    for jsonObj in f:
        studentDict = json.loads(jsonObj)
        studentsList.append(studentDict)

JSONDecodeError: Extra data: line 2 column 1 (char 102)

In [ ]:
def append_file(file_name, data):
    with open(file_name, 'a') as f:
        f.write(json.dumps(data, ensure_ascii=False)+ '\n')

# 2.push

In [86]:
#if 有檔案 else crawl

In [52]:
start_date = '0304'
end_date = '0305'

In [53]:
import os
flag = 0
if os.path.isfile("all_article.jsonl")== False:
    flag=1
    #crawl

In [54]:
url_push = []
with jsonlines.open('all_article.jsonl') as reader:
    for obj in reader:
        #print(obj['date']>=start_date and obj['date']<=end_date)
        if obj['date']>=start_date and obj['date']<=end_date:
            url_push.append(obj['url'])

In [56]:
good = []
bad = []
i=0
for url in url_push:
    print(i)
    i+=1
    res = reg.get(url)
    content = res.text
    soup = BeautifulSoup(content, "html.parser")
    comments = soup.find_all(class_ = "push")
    for comment in comments:
        bads = comment.find_all(class_ = "f1 hl push-tag")
        goods = comment.find_all(class_ = "hl push-tag")
        user = comment.find_all(class_ = "f3 hl push-userid")[0].text
        if len(bads)==1 and bads[0].text == '噓 ':
            bad.append(user)
        elif len(goods)==1 and goods[0].text == '推 ':
            good.append(user)
    # d = random.randint(1,4)
    # time.sleep(d)
print(good)
print(bad)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
['david0928', 'onegoman', 'takechance', 'TWilde', 'saatchi', 'david0928', 'Sessyoin', 'willx', 'relaxcloud', 'musichong100', 'asdfg5435', 'Aggro', 'mingdao', 'relaxcloud', 'Whitejjjj', 'jay228', 'LLSGG', 'GoogleTalk', 'Zensj1023', 'qscxz', 'smallkop', 'dolphin24681', 'tttggg', 'aaaaaaaaaa', 'tibo96033', 'keny80206', 'honglu587654', 'bread220', 'Alllllogo', 'len53', 'gtaped1997', 'morgen', 'neperstock', 'aterthy', 'johnnyham', 'nail5566', 'cady69', 'youwilldie', 'TitanEric', 'nail5566', 'Owl123', 'Justec', 'elfindor', 'qqwwweee', 'patrickchan', 'n94ap6999', 'wuyiyi', 'playerunknow', 'STRO', 'ppc', 'trenteric', 'Fluorescence', 'NEX4036', 'touchbird', 'crazytaxi123', 'sakasiaga', 'qaz13145204', 'iloserSD', 'hsiangyun', 'hiimwhisper', 'shau7276', 'tentaikanso', 'marverick', 'kyova', 'naloer', 'Yooooooooo', 'johnwu', 'sunnyyoung', 'bruce78131', 'seal998', 'shau7276', 'Bartolomeo', 'CmPunk666', 'karta328', 'ss

In [57]:
good_dict = {}
for g in set(good):
    good_dict[g] = good.count(g)

bad_dict = {}
for b in set(bad):
    bad_dict[b] = bad.count(b)


In [58]:
good_list = sorted(good_dict.items(),key=lambda x: (x[1],x[0][0]),reverse = True)
bad_list = sorted(good_dict.items(),key=lambda x: (x[1],x[0][0]),reverse = True)

In [62]:
push_ans= {
    "all_like": len(good),
    "all_boo": len(bad),
    "like 1":{"user_id": good_list[1][0], "count":good_list[1][1] }, 
    "like 2":{"user_id": good_list[2][0], "count":good_list[2][1] }, 
    "like 3":{"user_id": good_list[3][0], "count":good_list[3][1] }, 
    "like 4":{"user_id": good_list[4][0], "count":good_list[4][1] }, 
    "like 5":{"user_id": good_list[5][0], "count":good_list[5][1] }, 
    "like 6":{"user_id": good_list[6][0], "count":good_list[6][1] }, 
    "like 7":{"user_id": good_list[7][0], "count":good_list[7][1] }, 
    "like 8":{"user_id": good_list[8][0], "count":good_list[8][1] }, 
    "like 9":{"user_id": good_list[9][0], "count":good_list[9][1] }, 
    "like 10":{"user_id": good_list[10][0], "count":good_list[10][1] },
    "boo 1":{"user_id": bad_list[1][0], "count":bad_list[1][1] },
    "boo 2":{"user_id": bad_list[2][0], "count":bad_list[2][1] },
    "boo 3":{"user_id": bad_list[3][0], "count":bad_list[3][1] },
    "boo 4":{"user_id": bad_list[4][0], "count":bad_list[4][1] },
    "boo 5":{"user_id": bad_list[5][0], "count":bad_list[5][1] },
    "boo 6":{"user_id": bad_list[6][0], "count":bad_list[6][1] },
    "boo 7":{"user_id": bad_list[7][0], "count":bad_list[7][1] },
    "boo 8":{"user_id": bad_list[8][0], "count":bad_list[8][1] },
    "boo 9":{"user_id": bad_list[9][0], "count":bad_list[9][1] },
    "boo 10":{"user_id": bad_list[10][0], "count":bad_list[10][1] },
}

json_object = json.dumps(push_ans, indent=4)
 
# Writing to sample.json
with open(f"test_push.json", "w") as outfile:
    outfile.write(json_object)

# 3.popular

In [35]:
start_date = '0101'
end_date = '1231'

In [32]:
url_pop = []
with jsonlines.open('all_non_pop_2.jsonl') as reader:
    for obj in reader:
        #print(obj['date']>=start_date and obj['date']<=end_date)
        if obj['date']>=start_date and obj['date']<=end_date:
            url_pop.append(obj['url'])
len(url_pop)

1549

In [25]:
image_url = []
count = 0
for url in url_pop:
    res = reg.get(url)
    print(f'{count}: {url}')
    count += 1
    content = res.text
    soup = BeautifulSoup(content, "html.parser")
    images = soup.find_all("a")
    for image in images:
        if image.text.find('https://i.') != -1 :
            if image.text.endswith('.jpeg') or image.text.endswith('.jpg') or image.text.endswith('.png') or image.text.endswith('.gif'):
                image_url.append(image.text)
    d = random.uniform(0.2, 0.5)
    time.sleep(d)

0: https://www.ptt.cc/bbs/Beauty/M.1648731539.A.AFC.html
1: https://www.ptt.cc/bbs/Beauty/M.1648733778.A.9CB.html
2: https://www.ptt.cc/bbs/Beauty/M.1648740226.A.C64.html
3: https://www.ptt.cc/bbs/Beauty/M.1648648560.A.C26.html
4: https://www.ptt.cc/bbs/Beauty/M.1648648740.A.6C6.html
5: https://www.ptt.cc/bbs/Beauty/M.1648648932.A.60C.html
6: https://www.ptt.cc/bbs/Beauty/M.1648652682.A.6C6.html
7: https://www.ptt.cc/bbs/Beauty/M.1648654296.A.56F.html
8: https://www.ptt.cc/bbs/Beauty/M.1648654730.A.9C0.html
9: https://www.ptt.cc/bbs/Beauty/M.1648654907.A.568.html
10: https://www.ptt.cc/bbs/Beauty/M.1648654992.A.991.html
11: https://www.ptt.cc/bbs/Beauty/M.1648693655.A.829.html
12: https://www.ptt.cc/bbs/Beauty/M.1648693899.A.B14.html
13: https://www.ptt.cc/bbs/Beauty/M.1648694235.A.75E.html
14: https://www.ptt.cc/bbs/Beauty/M.1648698404.A.65B.html
15: https://www.ptt.cc/bbs/Beauty/M.1648724006.A.285.html
16: https://www.ptt.cc/bbs/Beauty/M.1648729043.A.B89.html
17: https://www.ptt.cc/b

In [33]:
#image_url2 = []
count = 0
for url in url_pop:
    res = reg.get(url)
    print(f'{count}: {url}')
    count += 1
    content = res.text
    soup = BeautifulSoup(content, "html.parser")
    images = soup.find_all("a")
    for image in images:
        if image.text.find('https://i.') != -1 :
            if image.text.endswith('.jpeg') or image.text.endswith('.jpg') or image.text.endswith('.png') or image.text.endswith('.gif'):
                image_url.append(image.text)
    d = random.uniform(0.2, 0.5)
    time.sleep(d)

0: https://www.ptt.cc/bbs/Beauty/M.1672465085.A.366.html
1: https://www.ptt.cc/bbs/Beauty/M.1672468987.A.B41.html
2: https://www.ptt.cc/bbs/Beauty/M.1672472517.A.28B.html
3: https://www.ptt.cc/bbs/Beauty/M.1672473850.A.908.html
4: https://www.ptt.cc/bbs/Beauty/M.1672477625.A.16E.html
5: https://www.ptt.cc/bbs/Beauty/M.1672488472.A.632.html
6: https://www.ptt.cc/bbs/Beauty/M.1672490236.A.BA4.html
7: https://www.ptt.cc/bbs/Beauty/M.1672380357.A.F26.html
8: https://www.ptt.cc/bbs/Beauty/M.1672381928.A.AFD.html
9: https://www.ptt.cc/bbs/Beauty/M.1672383666.A.2FD.html
10: https://www.ptt.cc/bbs/Beauty/M.1672383748.A.266.html
11: https://www.ptt.cc/bbs/Beauty/M.1672386976.A.2EE.html
12: https://www.ptt.cc/bbs/Beauty/M.1672391100.A.121.html
13: https://www.ptt.cc/bbs/Beauty/M.1672391947.A.7A3.html
14: https://www.ptt.cc/bbs/Beauty/M.1672400125.A.3C6.html
15: https://www.ptt.cc/bbs/Beauty/M.1672436301.A.E9B.html
16: https://www.ptt.cc/bbs/Beauty/M.1672452910.A.A9E.html
17: https://www.ptt.cc/b

In [34]:
len(image_url)

42015

In [36]:
non_pop_dict = {
    
    "number_of_popular_articles":len(url_pop),
    "image_urls": image_url, 
    
}

json_object = json.dumps(non_pop_dict, indent=4)

with open(f"2_non_popular_{start_date}_{end_date}.json", "w") as outfile:
    outfile.write(json_object)

# 4.keyword

In [327]:
keyword = "幽默"

In [315]:
url_key = []
with jsonlines.open('all_article.jsonl') as reader:
    for obj in reader:
        #print(obj['date']>=start_date and obj['date']<=end_date)
        if obj['date']>=start_date and obj['date']<=end_date:
            url_key.append(obj['url'])

In [316]:
len(url_key)

456

In [324]:
res = reg.get("https://www.ptt.cc/bbs/Beauty/M.1677567913.A.925.html")
content = res.text
soup = BeautifulSoup(content, "html.parser")
content = soup.find_all(class_ = "bbs-screen bbs-content")

In [325]:
content

[<div class="bbs-screen bbs-content" id="main-content"><div class="article-metaline"><span class="article-meta-tag">作者</span><span class="article-meta-value">airgp32002 (airgp32002)</span></div><div class="article-metaline-right"><span class="article-meta-tag">看板</span><span class="article-meta-value">Beauty</span></div><div class="article-metaline"><span class="article-meta-tag">標題</span><span class="article-meta-value">[正妹] 去澳門玩</span></div><div class="article-metaline"><span class="article-meta-tag">時間</span><span class="article-meta-value">Tue Feb 28 15:05:11 2023</span></div>
 <a href="https://i.imgur.com/RTob0gl.jpg" rel="noreferrer noopener nofollow" target="_blank">https://i.imgur.com/RTob0gl.jpg</a>
 <div class="richcontent"><img alt="" loading="lazy" src="https://cache.ptt.cc/c/https/i.imgur.com/RTob0gll.jpg?e=1677947230&amp;s=ZBy3h5vKXBFdgwMuUVYZHg"/></div><a href="https://i.imgur.com/HTxqyhl.jpg" rel="noreferrer noopener nofollow" target="_blank">https://i.imgur.com/HTxqyhl

In [310]:
import re
pattern = 'https:.*\.(?:jpg|gif|png|jpeg)'
re.findall(pattern, content[0].text)

['https://i.imgur.com/RTob0gl.jpg',
 'https://i.imgur.com/HTxqyhl.jpg',
 'https://i.imgur.com/sRZgejJ.jpg',
 'https://i.imgur.com/vqoA3OI.jpg']

In [332]:
keyword = "名字"
key_ans=[]
for url in url_key:
    #print(url)
    res = reg.get(url)
    content = res.text
    soup = BeautifulSoup(content, "html.parser")
    content = soup.find_all(class_ = "bbs-screen bbs-content")
    if keyword in content[0].text.split('發信站')[0]:
        print(url)
        main_content = content[0].text
        pattern = '(https?:\/\/.*?\.(?:png|jpg|gif|jepg))'
        imgs = re.findall(pattern, main_content)
        for img in imgs:
            #print(img)
            key_ans.append(img)
    else:
        d = random.randint(0,2)
        time.sleep(d)

https://www.ptt.cc/bbs/Beauty/M.1647669881.A.8D3.html
https://www.ptt.cc/bbs/Beauty/M.1647617243.A.019.html
https://www.ptt.cc/bbs/Beauty/M.1647533264.A.129.html
https://www.ptt.cc/bbs/Beauty/M.1647446576.A.EAB.html
https://www.ptt.cc/bbs/Beauty/M.1647153698.A.0AA.html
https://www.ptt.cc/bbs/Beauty/M.1646655502.A.345.html
https://www.ptt.cc/bbs/Beauty/M.1646464074.A.D0F.html


In [335]:
key_dict = {

    "image_urls": key_ans, 
    
}

json_object = json.dumps(key_dict, indent=4)

with open(f"keyword_{keyword}_{start_date}_{end_date}.json", "w") as outfile:
    outfile.write(json_object)